In [ ]:
import pandas as pd

In [ ]:
infos_ej = pd.read_xml('données/FINESS/simple/finess_cs1400101_stock_20211214-0333.xml.gz')

In [ ]:
infos_ej.head()

In [ ]:
infos_ej.columns

In [ ]:
colonnes_attendues = ['adresseacheminement', 'adressenumérovoie', 'adressetypevoie', 'adressevoie',
                      'libelléstatutjuridique', 'numérofinessentitéjuridique', 'raisonsociale', 'téléphone']

In [ ]:
infos_ej[['nofiness', 'rs', 'ligneacheminement', 'numvoie', 'typvoie', 'voie', 'libstatutjuridique', 'telephone']]

In [ ]:
table_cible_avec_valeurs_manquantes = infos_ej[
    ['nofiness', 'rs', 'ligneacheminement', 'numvoie', 'typvoie', 'voie', 'libstatutjuridique', 'telephone']
].rename(
    columns = {
        'nofiness':'numérofinessentitéjuridique', 
        'rs':'raisonsociale',
        'ligneacheminement': 'adresseacheminement',
        'numvoie': 'adressenumérovoie',
        'typvoie': 'adressetypevoie',
        'voie': 'adressevoie',
        'libstatutjuridique': 'libelléstatutjuridique',
        'telephone': 'téléphone'
    }
)

In [ ]:
table_cible_avec_finess_renseigné = table_cible_avec_valeurs_manquantes.dropna(
    subset=['numérofinessentitéjuridique']
)

In [ ]:
table_cible = table_cible_avec_finess_renseigné.fillna('').set_index('numérofinessentitéjuridique')